In [1]:
import numpy as np
from tensorflow_weave_help import *
from cython_weave import *

/Users/Tim/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
weave_params = {'num_zeros':2, 'filter_size':3}
#I_array_100_40_32_32_3 = create_full_I_array_weave_matrix((100,40,32,32), 10, weave_params)

In [3]:
def create_full_I_array_weave_matrix_fast(input_shape, max_num_images, weave_param):
    """
    createses properly sized 5D indexing tensor, I_array_weave that can be applied to
    tf.gather_nd(x, I_array_weave) that will return a tensor that is identical to
    x, cache = array_weave_fast_forward(tensor, num_zeros = 2)

    However, this takes a while to build in favor of fast execution time in the graph
    thus

    Creates an matrix of size (num_images, num_filters, height, width, input_tensor_dimension (4))
    """
    num_images,num_filters,num_rows,num_cols = input_shape
    num_images = max(max_num_images, num_images)

    tensor_dimension = len(input_shape)
    if tensor_dimension != 4:
        raise ValueError('Must operate on 4D tensors')
    if num_rows != num_cols:
        raise ValueErrro('Must operate ona square image')
    
    N_arr = np.arange(1,num_cols**2 + 1).reshape(
        [num_rows,num_cols]).astype(float)
    N_weave,_ = array_weave_fast_forward(N_arr, weave_param)
    num_rows, num_cols = N_weave.shape
        
    N_weave[np.where(N_weave == 0)] = -1
    print(num_cols//(weave_param['num_zeros']+1))
    
    I_temp = np.zeros([num_rows,num_cols,tensor_dimension])
    for i in range(num_cols):
        for j in range(num_rows):
            i_j_loc = np.where(N_weave[i,j] == N_arr)
            if len(i_j_loc[0]) > 0:
                I_temp[i,j] = np.array([-2,-3,i_j_loc[0],i_j_loc[1]])
            else:
                I_temp[i,j] = np.array([0,0,0,num_cols//(weave_param['num_zeros'])])
    
    I_full = np.zeros([num_images,num_filters,num_rows,num_cols,tensor_dimension])
    
    for img in range(num_images):
        for layer in range(num_filters):
            I_temp_current = np.copy(I_temp)
            I_temp_current[np.where(I_temp_current == -2)] = img
            I_temp_current[np.where(I_temp_current == -3)] = layer
            I_full[img,layer] = I_temp_current
                        
    return I_full.astype(int)

def create_full_I_zero_weave_matrix_fast(input_shape, max_num_images, weave_param):
    """
    createses properly sized 5D indexing tensor, I_array_weave that can be applied to
    tf.gather_nd(x, I_array_weave) that will return a tensor that is identical to
    x, cache = array_weave_fast_forward(tensor, num_zeros = 2)

    However, this takes a while to build in favor of fast execution time in the graph
    thus

    Creates an matrix of size (num_images, num_filters, height, width, input_tensor_dimension (4))
    """
    num_images,num_filters,num_rows,num_cols = input_shape
    num_images = max(max_num_images, num_images)

    tensor_dimension = len(input_shape)
    if tensor_dimension != 4:
        raise ValueError('Must operate on 4D tensors')
    if num_rows != num_cols:
        raise ValueErrro('Must operate ona square image')
    
    N_arr = np.arange(1,num_cols**2 + 1).reshape(
        [num_rows,num_cols]).astype(float)
    N_weave,_ = zero_weave_fast_forward(N_arr, weave_param)
    num_rows, num_cols = N_weave.shape
        
    N_weave[np.where(N_weave == 0)] = -1
    
    I_temp = np.zeros([num_rows,num_cols,tensor_dimension])
    for i in range(num_cols):
        for j in range(num_rows):
            i_j_loc = np.where(N_weave[i,j] == N_arr)
            if len(i_j_loc[0]) > 0:
                I_temp[i,j] = np.array([-2,-3,i_j_loc[0],i_j_loc[1]])
            else:
                I_temp[i,j] = np.array([0,0,0,num_cols//(weave_param['num_zeros'])])
    
    I_full = np.zeros([num_images,num_filters,num_rows,num_cols,tensor_dimension])
    
    for img in range(num_images):
        for layer in range(num_filters):
            I_temp_current = np.copy(I_temp)
            I_temp_current[np.where(I_temp_current == -2)] = img
            I_temp_current[np.where(I_temp_current == -3)] = layer
            I_full[img,layer] = I_temp_current
                        
    return I_full.astype(int)

In [4]:
A = create_full_I_array_weave_matrix_fast((4,4,5,5),4,weave_params)
B = create_full_I_array_weave_matrix((4,4,5,5), 4, weave_params)
print(np.mean(A== B))

4
0
1
2
3
1.0


In [5]:
A = create_full_I_zero_weave_matrix_fast((4,4,5,5),4,weave_params)
B = create_full_I_zero_weave_matrix((4,4,5,5), 4, weave_params)
print(np.mean(A== B))
print(A[1,0,0])
print(B[1,0,0])

0
1
2
3
1.0
[[1 0 0 0]
 [0 0 0 6]
 [0 0 0 6]
 [1 0 0 1]
 [0 0 0 6]
 [0 0 0 6]
 [1 0 0 2]
 [0 0 0 6]
 [0 0 0 6]
 [1 0 0 3]
 [0 0 0 6]
 [0 0 0 6]
 [1 0 0 4]]
[[1 0 0 0]
 [0 0 0 6]
 [0 0 0 6]
 [1 0 0 1]
 [0 0 0 6]
 [0 0 0 6]
 [1 0 0 2]
 [0 0 0 6]
 [0 0 0 6]
 [1 0 0 3]
 [0 0 0 6]
 [0 0 0 6]
 [1 0 0 4]]


In [6]:
A = create_full_I_zero_weave_matrix_fast((128,50,32,32),128,weave_params)